# Sentinel Download

In [2]:
# libraries import
import ee
import ipywidgets
import datetime
import ipywidgets as widgets
import ipyleaflet  # an interactive mapping "widget"
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [3]:
ee.Initialize()

In [4]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [5]:
vis_params = {
    'bands': ['HV'],
    'min':-20,
    'max':0,
}

In [6]:
roi_dimension = widgets.IntSlider(
    value=1e4,
    min=1e2,
    max=2e4,
    description='ROI Size (m):',
    continuous_update=False,
)

In [7]:
# Define the map.
map1 = ipyleaflet.Map(
    center=(81.736, -45.912),
    zoom=8,
    layout={'height':'600px'},
)
map1.add_control(ipyleaflet.LayersControl())

# Define and add a Marker pin to the map.
center_marker = ipyleaflet.Marker(
    name='ROI Selection Marker',
    location=map1.center
)
map1 += center_marker

mosaic_layer_group = ipyleaflet.LayerGroup(layers=(), name='Mosaic Layer')
map1 += mosaic_layer_group

roi_layer_group = ipyleaflet.LayerGroup(layers=(), name='ROI Layer')
map1 += roi_layer_group

In [8]:
start_datepicker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime.datetime(2017, 8, 1)
)
end_datepicker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime.datetime(2017, 8, 10)
)

In [9]:
out = widgets.Output()

In [10]:
# Layout the UI elements.
panel = widgets.VBox([
    map1,
    start_datepicker,
    end_datepicker,
    roi_dimension,
    out
])

out.clear_output()
display(panel)

In [11]:
# Define helper functions to swap the coordinate ordering.
def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0])

def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]  

def update_roi_layer(map_reference):    
    coord_list_xy = get_roi_polygon()['coordinates'][0]
    coord_list_yx = swap_coordinate_xy_for_list(coord_list_xy)
    
    roi_layer = ipyleaflet.Polygon(
        name='TEST update ROI Polygon',
        locations=coord_list_yx,
        weight=3,
        color='#F00',
        opacity=0.8,
        fill_opacity=0.1,
        fill_color='#F00'
    )
    roi_layer_group.clear_layers()
    roi_layer_group.add_layer(roi_layer)

In [12]:
def get_roi_polygon():
    center_marker_xy = swap_coordinate_xy_for_location(center_marker.location)
    centroid = ee.Geometry.Point(center_marker_xy)
    buffered = centroid.buffer(roi_dimension.value).bounds()
    return buffered.getInfo()

In [13]:
def get_image_collection():

    # Get filter values from the UI widgets.
    roi = get_roi_polygon()
    start_date = ee.Date(start_datepicker.value.isoformat())
    end_date = ee.Date(end_datepicker.value.isoformat())

    collection = (
        base_collection
          .filterDate(start_date, end_date)
          .filterBounds(roi)
    )
    return collection

In [14]:
base_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
      .select(['HV'])
)

In [15]:
def update_mosaic_layer(map_reference):
    
    out.clear_output()
    with out:
        print('Total images = {0}'.format(get_image_collection().size().getInfo()))
    
    mosaic_tilelayer = ipyleaflet.TileLayer(
        url=GetTileLayerUrl(
            get_image_collection().mosaic().visualize(**vis_params)
        ),
        attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
    )
    mosaic_layer_group.clear_layers()
    mosaic_layer_group.add_layer(mosaic_tilelayer)

In [16]:
# Define the actions performed when the marker moves.
def center_marker_on_move(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
center_marker.unobserve_all()
center_marker.observe(center_marker_on_move, names='location')

In [17]:
# Define the actions performed when the ROI size is changed.
def roi_dimension_on_change(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
roi_dimension.unobserve_all()
roi_dimension.observe(roi_dimension_on_change, names='value')

In [18]:
update_roi_layer(map1)
update_mosaic_layer(map1)

Total images = 15


In [19]:
collection = get_image_collection()
print(collection.size().getInfo())

15


In [20]:
MAX_ELEMENTS=100
images = collection.toList(MAX_ELEMENTS).getInfo()

In [21]:
[img['id'] for img in images]

['COPERNICUS/S1_GRD/S1B_EW_GRDH_1SDH_20170801T115853_20170801T115953_006746_00BDEC_CD24',
 'COPERNICUS/S1_GRD/S1B_EW_GRDH_1SDH_20170803T114230_20170803T114330_006775_00BEC0_0FB5',
 'COPERNICUS/S1_GRD/S1B_EW_GRDH_1SDH_20170805T112608_20170805T112708_006804_00BF90_D5E8',
 'COPERNICUS/S1_GRD/S1B_EW_GRDH_1SDH_20170806T120705_20170806T120805_006819_00C005_F47A',
 'COPERNICUS/S1_GRD/S1B_EW_GRDH_1SDH_20170808T115042_20170808T115142_006848_00C0DF_58E7',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170802T110130_20170802T110213_006760_00BE54_8FE2',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170803T181419_20170803T181524_006779_00BEDA_4205',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170804T104513_20170804T104613_006789_00BF1F_FF61',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170805T175757_20170805T175901_006808_00BFA7_A5A1',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170806T102844_20170806T102919_006818_00BFFD_F715',
 'COPERNICUS/S1_GRD/S1B_EW_GRDM_1SDH_20170806T183851_20170806T183955_006823_00C024_2873',
 'COPERNIC